In [11]:
    import requests
    import pandas as pd
    from datetime import date
    import numpy as np

    # URL template (corrected)
    url = "https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15"

    # Instrument tokens
    tickerData = {"NIFTY": {"id": 256265}, "BANKNIFTY": {"id": 260105}}

    # Interval mapping
    KI = {
        "3m": "3minute",
        "5m": "5minute",
        "15m": "15minute",
        "60m": "60minute",
        "day": "day",
    }

    # Function to get candle data
    def getCandles(symbol, fromDate, toDate, timeframe):
        headers = {
            "authorization": "enctoken l9RBs2mUaCOtHh76/Al766xp7yJ1ndunq9zE2PslZvpMEXu0ccRXlWhWHNUOyJX19msbiNnAcd37PdO0yb1ikRiT3q1N7v2hutAK2FB3s2GAGxkfVgusyQ=="
            # Replace this with your actual token securely
        }
        curUrl = url.format(tickerData[symbol]["id"], timeframe, fromDate, toDate)
        print("Requesting:", curUrl)

        try:
            session = requests.session()
            response = session.get(curUrl, headers=headers)
            r = response.json()
        except Exception as error:
            print("API request failed:", error)
            return pd.DataFrame()

        try:
            history = r["data"]["candles"]
            history_df = pd.DataFrame(history)
            history_df = history_df.rename(
                columns={
                    0: "Datetime",
                    1: "open",
                    2: "high",
                    3: "low",
                    4: "close",
                    5: "volume",
                }
            )
            history_df["Datetime"] = pd.to_datetime(history_df["Datetime"])
            history_df.set_index("Datetime", inplace=True)
            return history_df
        except Exception as error:
            print("Data processing failed:", error)
            return pd.DataFrame()


    # Marubozu pattern detector
    def maru_bozu(ohlc_df):
        """Returns dataframe with marubozu candle pattern label"""
        df = ohlc_df.copy()
        avg_candle_size = abs(df["close"] - df["open"]).median()

        # Upper and lower wicks
        df["h-c"] = df["high"] - df["close"]
        df["l-o"] = df["low"] - df["open"]
        df["h-o"] = df["high"] - df["open"]
        df["l-c"] = df["low"] - df["close"]

        # Green Marubozu
        condition_green = (df["close"] - df["open"] > 2 * avg_candle_size) & (
            df[["h-c", "l-o"]].abs().max(axis=1) < 0.003 * avg_candle_size
        )

        # Red Marubozu
        condition_red = (df["open"] - df["close"] > 2 * avg_candle_size) & (
            df[["h-o", "l-c"]].abs().max(axis=1) < 0.003 * avg_candle_size
        )

        df["maru_bozu"] = np.where(
            condition_green,
            "maru_bozu_green",
            np.where(condition_red, "maru_bozu_red", False),
        )

        df.drop(["h-c", "l-o", "h-o", "l-c"], axis=1, inplace=True)
        return df


    print("Start")
    symbol_data = getCandles("NIFTY", "2025-03-20", "2025-03-25", KI["15m"])


    if not symbol_data.empty:
        ma_df = maru_bozu(symbol_data)
        print(ma_df[ma_df["maru_bozu"] != False])
    else:
        print("No OHLC data available.")


Start
Requesting: https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15
                               open      high       low     close  volume  6  \
Datetime                                                                       
2025-06-11 09:15:00+05:30  25134.15  25136.20  25081.30  25128.20       0  0   
2025-06-11 09:20:00+05:30  25127.60  25137.55  25115.05  25119.90       0  0   
2025-06-11 09:25:00+05:30  25120.30  25137.00  25117.70  25136.65       0  0   
2025-06-11 09:30:00+05:30  25135.95  25144.05  25128.40  25141.85       0  0   
2025-06-11 09:35:00+05:30  25141.65  25150.40  25122.35  25125.10       0  0   
...                             ...       ...       ...       ...     ... ..   
2025-06-13 15:05:00+05:30  24702.60  24727.90  24702.60  24727.90       0  0   
2025-06-13 15:10:00+05:30  24727.95  24728.45  24717.20  24725.60       0  0   
2025-06-13 15:15:00+05:30  24725.15  24725.90  24709.75  24717.95